In [1]:
import tweepy
from TweetTransformations import checkClaims, translate_to_german,  translate_to_english, deconstructTwitterQueryResponse
from config import access_token2, access_token_secret2, bearer_token2, consumer_key2, consumer_secret2


In [2]:
api = tweepy.Client(
    access_token=access_token2,
    access_token_secret=access_token_secret2,
    bearer_token=bearer_token2,
    consumer_key=consumer_key2,
    consumer_secret=consumer_secret2,
    wait_on_rate_limit=False)

In [3]:
# keyword search for klimawandel 
response = api.search_recent_tweets(query="Klimawandel -is:retweet -is:reply", 
                                    expansions=["author_id",'referenced_tweets.id'],
                                    tweet_fields=["reply_settings"],
                                    user_fields=["protected"],
                                    max_results=20)

In [4]:
response.data[0].id

1952762285304254579

In [5]:
tweet_list = deconstructTwitterQueryResponse(response)

OK to reply to Tweet 1952762285304254579
OK to reply to Tweet 1952760292187759087
OK to reply to Tweet 1952759024862023867
OK to reply to Tweet 1952758937855349072
OK to reply to Tweet 1952758209124384882
OK to reply to Tweet 1952757778080055586
OK to reply to Tweet 1952753811912532004
Skipping Tweet 1952753707138826724: Replies are limited to 'following'.
OK to reply to Tweet 1952749288192090160
OK to reply to Tweet 1952749170860666927
OK to reply to Tweet 1952748994783814051
OK to reply to Tweet 1952748696287756296
OK to reply to Tweet 1952748320130023870
OK to reply to Tweet 1952747799608524828
OK to reply to Tweet 1952747372389208458
OK to reply to Tweet 1952747158001639573
OK to reply to Tweet 1952744297301672054
OK to reply to Tweet 1952742599435174339
OK to reply to Tweet 1952740157209157683
OK to reply to Tweet 1952740103790510250


In [ ]:
response.data

In [ ]:
list_of_tweets = []
for tweet in response.data:
    # CORRECTED LOGIC: Check the 'reply_settings' attribute directly
    if tweet.reply_settings != 'everyone':
        print(f"Skipping Tweet {tweet.id}: Replies are limited to '{tweet.reply_settings}'.")
        continue  # Skip to the next tweet
    
    # If the check passes, you can process or add the tweet
    print(f"OK to reply to Tweet {tweet.id}")
    tweet_id = tweet.id
    original_tweet = tweet.text
    answer_dict = {
            "original_tweet" : original_tweet,
            "tweet_id": tweet_id
        }
    list_of_tweets.append(answer_dict)
    

In [ ]:
list_of_tweets

In [ ]:
tweet_list = list_of_tweets

In [6]:
translated_tweet_list = []
for tweet in tweet_list:
    tweet_id = tweet.get("tweet_id")
    original_tweet= tweet.get("original_tweet")
    translated_original_tweet = await translate_to_english(original_tweet)
    answer_dict = {
                "original_tweet" : original_tweet,
                "tweet_id": tweet_id,
                "translated_original_tweet" : translated_original_tweet,
            }
    translated_tweet_list.append(answer_dict)

In [8]:
translated_tweet_list

[{'original_tweet': 'SgH @lawen4cer, das ist eine Milchmädchenrechnung. Von den 30% Anstieg sind max. 5% vom Menschen = 1,5%. Zudem hatten wir vor 450 Mio Jahren einen #CO2 Anteil von 2.000 ppm bei sinkenden #Temperaturen👇.  Menschen gibt es aber erst seit 2,8 Mio Jahren. #Klimawandel https://t.co/wPDcRphfGj https://t.co/Pj9b0e5sR0',
  'tweet_id': 1952762285304254579,
  'translated_original_tweet': "SgH @lawen4cer, that's a childish calculation. Of the 30% increase, a maximum of 5% is from humans = 1.5%. Moreover, we had a #CO2 level of 2,000 ppm 450 million years ago with falling #temperatures👇. Humans have only existed for 2.8 million years. #ClimateChange"},
 {'original_tweet': '"#HAARP wurde dafür konzipiert, Energie in die Atmosphäre einzuspeisen, um sie zu kontrollieren..."\n\n#HAARP + das tägliche Spray + #Schumann_Frequenzwellen = #Klimawandel https://t.co/XB8hALvonR',
  'tweet_id': 1952760292187759087,
  'translated_original_tweet': '#HAARP was designed to inject energy into t

In [7]:
list_of_factchecked_tweets = await checkClaims(tweetlist=translated_tweet_list)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./VectorStorage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./VectorStorage/index_store.json.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: RAG_Lookup_tool
Action Input: {"input": "Claims in the tweet: 1) 30% increase is from humans, maximum of 5% is from humans = 1.5%. 2) CO2 level of 2,000 ppm 450 million years ago with falling temperatures. 3) Humans have only existed for 2.8 million years."}
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The tweet contains several inaccuracies. Firstly, the claim that only 5% of a 30% increase is from humans is misleading. While natural factors influence climate, the scientific consensus is that human activities are the dominant driver of the current warming trend. Secondly, while CO2 levels were high 450 million years ago, the climate s

In [10]:
len(list_of_factchecked_tweets)

12

In [12]:
from llama_index.llms.ollama import Ollama
llm = Ollama(
    model= "gemma3:12b",
    request_timeout=120.0,
    context_window=8128,
    temperature=0.0
)

In [14]:
for tweet in list_of_factchecked_tweets:
    original_tweet = tweet.get("original_tweet")
    answer_tweet = tweet.get("answer_tweet")
    tweet_id = tweet.get("tweet_id")
    translated_answer_tweet = await translate_to_german(answer_tweet)
    shortened_tweet = await llm.acomplete(f"Dies ist ein Tweet. Dieser ist jedoch deutlich zu lang. Kürze den tweet auf so kurz wie möglich. ab besten unter 30 Wörter. Halte dich dabei umgangssprachlich {translated_answer_tweet}")
    print(tweet_id)
    # post tweet
    try:
        api.create_tweet(text=shortened_tweet.text,in_reply_to_tweet_id=tweet_id)
    except tweepy.errors.Forbidden as e:
        print(f"tried to write {translated_answer_tweet} to {tweet_id}")
        print(f"error while posting: {e}")


1952762285304254579
1952760292187759087
1952758937855349072
1952758209124384882
1952757778080055586
1952753811912532004


TooManyRequests: 429 Too Many Requests
Too Many Requests